# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  5.28it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.69it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.35it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.25it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.39it/s]



  0%|          | 0/23 [00:00<?, ?it/s]

...

 83%|████████▎ | 19/23 [00:05<00:01,  3.51it/s]

 87%|████████▋ | 20/23 [00:05<00:00,  3.81it/s]

 91%|█████████▏| 21/23 [00:06<00:00,  4.02it/s]

 96%|█████████▌| 22/23 [00:06<00:00,  4.22it/s]

100%|██████████| 23/23 [00:06<00:00,  3.56it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Diana, and I am the principal of Pacific High School. I am thrilled to welcome you to our school community! As the principal, I am committed to providing a safe, supportive, and inclusive environment where our students can thrive academically, socially, and emotionally. Our school is dedicated to fostering a growth mindset, promoting social-emotional learning, and encouraging students to become responsible, compassionate, and engaged members of our community. At Pacific High School, we believe that every student has unique talents, interests, and abilities that we strive to develop and nurture. We aim to provide our students with a well-rounded education that prepares them for success
Prompt: The president of the United States is
Generated text:  the head of state and head of government of the United States, and is the highest public official in the country. The president serves as the commander-in-chief of the armed forces and is responsible 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 K

 Kelsey

 Kelsey,

 Kelsey, and

...

 all about collaboration between humans and machines, according to Dr. Gary Marcus, CEO of Robust.AI, and a pioneer in the field of AI and robotics.
Marcus recently spoke at the World Economic Forum in Davos, where he emphasized the importance of collaboration between humans and AI systems in order to unlock their full potential. He noted that while AI systems are becoming increasingly powerful, they still lack the common sense and critical thinking abilities of humans, and that they need to be designed to work in conjunction with humans in order to achieve optimal results.
Marcus has a background in cognitive psychology and has written several books on the subject of

 all about collaboration between humans and machines, according to Dr. Gary Marcus, CEO of Robust.AI, and a pioneer in the field of AI and robotics.
Marcus recently spoke at the World Economic Forum in Davos, where he emphasized the importance of collaboration between humans and AI systems in order to unlock their full potential. He noted that while AI systems are becoming increasingly powerful, they still lack the common sense and critical thinking abilities of humans, and that they need to be designed to work in conjunction with humans in order to achieve optimal results.
Marcus has a background in cognitive psychology and has written several books on the subject of AI

 all about collaboration between humans and machines, according to Dr. Gary Marcus, CEO of Robust.AI, and a pioneer in the field of AI and robotics.
Marcus recently spoke at the World Economic Forum in Davos, where he emphasized the importance of collaboration between humans and AI systems in order to unlock their full potential. He noted that while AI systems are becoming increasingly powerful, they still lack the common sense and critical thinking abilities of humans, and that they need to be designed to work in conjunction with humans in order to achieve optimal results.
Marcus has a background in cognitive psychology and has written several books on the subject of AI,

 all about collaboration between humans and machines, according to Dr. Gary Marcus, CEO of Robust.AI, and a pioneer in the field of AI and robotics.
Marcus recently spoke at the World Economic Forum in Davos, where he emphasized the importance of collaboration between humans and AI systems in order to unlock their full potential. He noted that while AI systems are becoming increasingly powerful, they still lack the common sense and critical thinking abilities of humans, and that they need to be designed to work in conjunction with humans in order to achieve optimal results.
Marcus has a background in cognitive psychology and has written several books on the subject of AI, including

 all about collaboration between humans and machines, according to Dr. Gary Marcus, CEO of Robust.AI, and a pioneer in the field of AI and robotics.
Marcus recently spoke at the World Economic Forum in Davos, where he emphasized the importance of collaboration between humans and AI systems in order to unlock their full potential. He noted that while AI systems are becoming increasingly powerful, they still lack the common sense and critical thinking abilities of humans, and that they need to be designed to work in conjunction with humans in order to achieve optimal results.
Marcus has a background in cognitive psychology and has written several books on the subject of AI, including "

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Shazad and I’m a strategy consultant with a passion for cricket and a love for helping businesses grow.
Here are some things you might find interesting about me:
I hold an MBA from the University of Oxford and a Bachelors in Business Administration from the University of California, Berkeley.
I have over 10 years of experience in strategy consulting, having worked with top-tier firms such as McKinsey & Company and Boston Consulting Group.
My areas of expertise include market entry strategies, portfolio optimization, organizational design, and digital transformation.
I have a deep understanding of the consumer goods, healthcare, and technology sectors, and have worked with clients across

Prompt: The capital of France is
Generated text:  famous for its iconic landmarks, rich history, and vibrant culture. Visitors can explore the Eiffel Tower, visit the Louvre Museum, and stroll along the Seine River. The city is also known for its fashion, cui

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Se

 Seung

 Seung J

 Seung Joon

...

 brighter than ever, with AI-powered systems transforming various industries and aspects of life. One of the most exciting and rapidly evolving areas of AI is in healthcare, where AI is being used to improve patient outcomes, streamline clinical workflows, and reduce healthcare costs. In this article, we will explore the exciting developments in AI-powered healthcare and what the future may hold.
AI-Powered Diagnostics
Artificial intelligence is revolutionizing the field of medical diagnostics by analyzing medical images, such as X-rays, CT scans, and MRIs, to help doctors identify diseases earlier and more accurately. AI algorithms can detect patterns and anomalies in medical

 brighter than ever, with AI-powered systems transforming various industries and aspects of life. One of the most exciting and rapidly evolving areas of AI is in healthcare, where AI is being used to improve patient outcomes, streamline clinical workflows, and reduce healthcare costs. In this article, we will explore the exciting developments in AI-powered healthcare and what the future may hold.
AI-Powered Diagnostics
Artificial intelligence is revolutionizing the field of medical diagnostics by analyzing medical images, such as X-rays, CT scans, and MRIs, to help doctors identify diseases earlier and more accurately. AI algorithms can detect patterns and anomalies in medical images

 brighter than ever, with AI-powered systems transforming various industries and aspects of life. One of the most exciting and rapidly evolving areas of AI is in healthcare, where AI is being used to improve patient outcomes, streamline clinical workflows, and reduce healthcare costs. In this article, we will explore the exciting developments in AI-powered healthcare and what the future may hold.
AI-Powered Diagnostics
Artificial intelligence is revolutionizing the field of medical diagnostics by analyzing medical images, such as X-rays, CT scans, and MRIs, to help doctors identify diseases earlier and more accurately. AI algorithms can detect patterns and anomalies in medical images that

 brighter than ever, with AI-powered systems transforming various industries and aspects of life. One of the most exciting and rapidly evolving areas of AI is in healthcare, where AI is being used to improve patient outcomes, streamline clinical workflows, and reduce healthcare costs. In this article, we will explore the exciting developments in AI-powered healthcare and what the future may hold.
AI-Powered Diagnostics
Artificial intelligence is revolutionizing the field of medical diagnostics by analyzing medical images, such as X-rays, CT scans, and MRIs, to help doctors identify diseases earlier and more accurately. AI algorithms can detect patterns and anomalies in medical images that may

 brighter than ever, with AI-powered systems transforming various industries and aspects of life. One of the most exciting and rapidly evolving areas of AI is in healthcare, where AI is being used to improve patient outcomes, streamline clinical workflows, and reduce healthcare costs. In this article, we will explore the exciting developments in AI-powered healthcare and what the future may hold.
AI-Powered Diagnostics
Artificial intelligence is revolutionizing the field of medical diagnostics by analyzing medical images, such as X-rays, CT scans, and MRIs, to help doctors identify diseases earlier and more accurately. AI algorithms can detect patterns and anomalies in medical images that may be

In [6]:
llm.shutdown()